# GeoCAT-viz Demo 1

## NCL_animate_1.py
__This script illustrates the following concepts:__
- Creating animations using matplotlib.FuncAnimation

__See following URLs to see the reproduced NCL plot & script:__
- Original NCL script: https://www.ncl.ucar.edu/Applications/Scripts/animate_1.ncl

- Original NCL plot: https://www.ncl.ucar.edu/Applications/Images/animate_1_1_lg.png

__Please note:__
- Executing this script will not display a gif, but you have the option to uncomment a line at the bottom that will save a gif in the same directory as this script.

### Imports

In [ ]:
import cartopy.crs as ccrs
import matplotlib.animation as animation
import numpy as np
import xarray as xr
from matplotlib import pyplot as plt

import geocat.datafiles as gdf
import geocat.viz as gv

### Data Import

In [ ]:
# Open a netCDF data file using xarray default engine and load the data into xarrays
# Disable time decoding due to missing necessary metadata
ds = xr.open_dataset(gdf.get("netcdf_files/meccatemp.cdf"), decode_times=False)

tas = ds.t

### Figure Setup

In [ ]:
fig = plt.figure(figsize=(10, 8))
# Generate axes using Cartopy and draw coastlines
ax = plt.axes(projection=ccrs.PlateCarree(central_longitude=150))
ax.coastlines(linewidths=0.5)
ax.set_extent([-180, 180, -90, 90], ccrs.PlateCarree())

# Use geocat.viz.util convenience function to set axes limits & tick values
gv.set_axes_limits_and_ticks(ax,
                             xlim=(-180, 180),
                             ylim=(-90, 90),
                             xticks=np.linspace(-180, 180, 13),
                             yticks=np.linspace(-90, 90, 7))

# Use geocat.viz.util convenience function to add minor and major tick lines
gv.add_major_minor_ticks(ax, labelsize=10)

# Use geocat.viz.util convenience function to make latitude, longitude tick labels
gv.add_lat_lon_ticklabels(ax)

# create initial plot that establishes a colorbar
tas[0, :, :].plot.contourf(ax=ax,
                           transform=ccrs.PlateCarree(),
                           vmin=195,
                           vmax=328,
                           levels=53,
                           cmap="inferno",
                           cbar_kwargs={
                               "extendrect": True,
                               "orientation": "horizontal",
                               "ticks": np.arange(195, 332, 9),
                               "label": "",
                               "shrink": 0.90
                           })

### Amination Function

In [ ]:
# animate function for matplotlib FuncAnimation
def animate(i):
    tas[i, :, :].plot.contourf(
        ax=ax,
        transform=ccrs.PlateCarree(),
        vmin=195,
        vmax=328,
        levels=53,
        cmap="inferno",
        add_colorbar=False,
    )

    gv.set_titles_and_labels(
        ax,
        maintitle="January Global Surface Temperature (K) - Day  " +
        str(tas.coords['time'].values[i])[:13],
        xlabel="",
        ylabel="")

### Animation Display

In [ ]:
# runs the animation initiated with the frame from init and progressed with the animate function
anim = animation.FuncAnimation(fig, animate, frames=30, interval=200)

### Save Animation

In [ ]:
# Uncomment this line to save the created animation
anim.save('animate_1.gif', writer='pillow', fps=5)